## **BẢNG PHÂN CHIA NHIỆM VỤ**

<table>
    <thead>
        <tr>
            <th rowspan=3, style ="text-align:center">MSSV</th>
            <th rowspan=3, style ="text-align:center">Họ tên</th>
            <th colspan=4, style ="text-align:center">Nhiệm vụ</th>
        </tr>
        <tr>
            <th rowspan=2, style ="text-align:center">Trình bày, báo cáo</th>
            <th colspan=3, style ="text-align:center">Câu hỏi</th>
            <th colspan=3, style ="text-align:center">Tiền xử lý dữ liệu</th>
            <th colspan=3, style ="text-align:center">Trực quan dữ liệu</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>19120684</td>
            <td>Trần Bảo Tín</td>
            <td style ="text-align:center">&check;</td>
            <td style ="text-align:center">&check;</td>
            <td style ="text-align:center">&check;</td>
            <td></td>
        </tr>
        <tr>
            <td>19120473</td>
            <td>Phạm Thành Đạt</td>
            <td style ="text-align:center">&check;</td>
            <td style ="text-align:center">&check;</td>
            <td></td>
            <td style ="text-align:center">&check;</td>
        </tr>
    </tbody>
</table>

# MỘT CHÚT LƯU Ý

Vì `chomedrive` trên mỗi hệ điều hành thì khác nhau. Nhưng không biết quý thầy cô chấm điểm trên hệ điều hành nào. Cho nên hàm `kind_os` sẽ giúp chọn drive hợp lí.

**Note:** Nhập loại hệ điều hành mà quý thầy cô sử dụng. Măc định: window
1. Window
2. Linux
3. MacOs (không phải chip M1)'''

In [1]:
kind_chromedrive = "./chromedriver.exe"

In [3]:
def kind_os():
      '''Nhập loại hệ điều hành mà quý thầy cô sử dụng. Măc định: linux
      1. Window
      2. Linux
      3. MacOs (không phải chip M1)'''
      kind_chromedrive = ""
      choose = input("Kind of OS: ")
      if(choose == "1"):
            kind_chromedrive = "./chromedriver.exe"
      elif(choose == "2"):
            kind_chromedrive = "./chromedriver_linux"
      elif(choose == "3"):
            kind_chromedrive = "./chromedriver_mac"
      return kind_chromedrive

In [4]:
kind_chromedrive = kind_os()

Kind of OS: 1


# IMPORT THƯ VIỆN

In [5]:
# impor thư viện
import requests
import requests_cache
from bs4 import BeautifulSoup
import re
import csv
import time
import json
import random
import pandas as pd
import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

# thư viện selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

# THU THẬP DỮ LIỆU CHO USER

--- 
# Setup toàn cục

Ở bước này sẽ setup các biến như:
- `sleep_time`. tạo 1 khoảng thời gian nghỉ khi requests hoặc đợi selenium thực hiện 1 vài tác vụ.
- `cache`
- `url`. Tạo đường dẫn trang chủ của website, sẽ được sử dụng toàn cục.
- `ChromeOptions`
- `key_user_file` . đường dẫn đến file chứa key cho việc search user
- `correct_user_file` . đường dẫn đến file `correct_user.csv` mà chương trình vừa mới tạo ra 
- `user_file` . đường dẫn đến file `user.csv` mà bọn em đã thu thập được

In [6]:
# setup cache
def filter_response(response) -> bool:
    return '"incomplete_results":false' in response.text 

In [7]:
# test 
requests_cache.install_cache('cache', backend ='sqlite', expire_after = 10000,filter_fn = filter_response)

In [8]:
url_homepage = 'https://soundcloud.com'

In [9]:
sleep_time = 1 # đơn vị giây

In [10]:
# setup connect device.
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [11]:
key_user = "./key_search/key_user.csv"

In [12]:
correct_user_file = "./correct_file/correct_user.csv"

In [13]:
user_file = "./HTML_data/user.csv"

---
# Các thao tác xử lí

## 1.Khai báo các hàm

file `key_username.csv` có chứa hơn 100 tên thông dụng trên thế giới. Đường dẫn khi tìm kiếm các user có dạng https://soundcloud.com/search/people?q= {name}. Trong đó name: là tên ta lấy từ file `key_username.csv`

In [14]:
def search_url_usernames(random_usernames):
    url_usernames = []
    f_reader = open(random_usernames,'r')
    csv_read = csv.reader(f_reader,delimiter = '\n')
    search_url = 'https://soundcloud.com/search/people?q='
    for row in csv_read:
        row = (str(row).rstrip(']')).lstrip('[')
        url_usernames.append(search_url + row)
    return url_usernames

In [18]:
# test
url_usernames = search_url_usernames('key_user.csv')

Dùng `selenium` để tìm kiếm user theo tên đã có. Với thuộc tính `href` thì sẽ có được đường dẫn vào trang cá nhân của `user` vì lúc tìm kiếm có thể tên `user` trùng nhau vì thế phải lọc lại bằng `set()` các đường dẫn, vì đường dẫn mỗi user là duy nhất.

Kết quả thu được là `set()` các đường dẫn không trùng nhau vào trang cá nhân của users

In [19]:
def collect_url_users(url_usernames):
    url_users = set()
    for url_username in url_usernames:
        if len(url_users) <= 1000 :
            driver = webdriver.Chrome(kind_chromedrive,options = options)
            driver.get(url_username)
            driver.implicitly_wait(1)
            soup = BeautifulSoup(driver.page_source,'lxml')
            driver.close()
            header_users = soup.find_all('a', {'class' : 'sc-link-dark sc-link-primary'})
            for header_user in header_users :
                url_users.add(url_homepage + header_user.get('href'))
        else : break
    return url_users

In [20]:
# test
url_users = collect_url_users(url_usernames)
url_users

{'https://soundcloud.com/fitzpatrickmusic',
 'https://soundcloud.com/pjmorton',
 'https://soundcloud.com/djnitinpatil',
 'https://soundcloud.com/00robertson',
 'https://soundcloud.com/user-84432806',
 'https://soundcloud.com/joanneclara',
 'https://soundcloud.com/user-9726483',
 'https://soundcloud.com/michael-mandal',
 'https://soundcloud.com/moshe-mosayev',
 'https://soundcloud.com/olivia-ungaro',
 'https://soundcloud.com/user-198085577',
 'https://soundcloud.com/mandal-forbes',
 'https://soundcloud.com/sanajitmandal',
 'https://soundcloud.com/sipama-parandzem-grigoryan',
 'https://soundcloud.com/ethan-278463209',
 'https://soundcloud.com/joanneworldtour',
 'https://soundcloud.com/noahofficialusa',
 'https://soundcloud.com/joeljesseanderson',
 'https://soundcloud.com/rasulh',
 'https://soundcloud.com/user-73527773',
 'https://soundcloud.com/catherine-bulaybulay-gelbero',
 'https://soundcloud.com/natkingcole',
 'https://soundcloud.com/kyle-dunnigan',
 'https://soundcloud.com/bramstoke

## 2. Xử lí chính

Thu thập dữ liệu hơn 1000 user bằng html-parse một số trường thuộc tính thu thập
1. url_user : đường dẫn trang cá nhân user
2. url_avatar: đường dẫn ảnh đại diện user
3. url_visual: đường dẫn hình nền user
4. url_station : đường dẫn đến kho nhạc của user
5. id_user: id của user
6. username: tên nghệ danh của user
7. name: tên thật của user
8. address: địa chỉ user gồm city và country 
9. verified: kiểm tra user đã xác minh 
10. pro_unlimited,pro : kiểm tra user có thuộc diện đặc biệt của website
11. description: mô tả về website do chính user viết
12. follower: người theo dõi user
13. following: người user đang theo dõi
14. track : bài hát được đăng của user
15. like: lượt thích bởi user
16. playlist : danh sách nhạc
17. ablum : album nhạc
18. repost: bài đăng của user
19. comment_count: số bình luận của user
    
**Lưu ý** :Các thuộc tính liên quan đến `playlist`, `album`, `repost` vì phải `scroll down` tới cuối trang nên mất khá nhiều thời gian và phụ thuộc vào `user` nên em có một `file correct_user` đã thu thập đầy đủ các trường thông tin trên để thầy/cô xem qua.

      Còn bài làm ở đây vì để tiết kiếm thời gian nên em không đưa vào.

In [42]:
def collect_information_users(url_users,users_file) :
    f_writer = open(users_file,'w', encoding =' utf-8')
    f_writer.write('url_user,url_avatar,id_user,username,verified,name,city,country,description,'\
        'pro_unlimited,pro,url_followers,followers_count,url_following,followings_count,'\
        'url_track,tracks_count,url_station,url_like,likes_count,url_visual,comments_count\n')
    for url_user in url_users :
        driver = webdriver.Chrome(kind_chromedrive, options = options)
        driver.get(url_user)
        try:
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CLASS_NAME, 'l-container l-content')))
        except TimeoutException:
            soup = BeautifulSoup(driver.page_source,'lxml')
            driver.close()
            row = url_user + ','
            
            # url_avater
            url_avatar = soup.find( 'div', {'class':'profileHeaderInfo sc-media'})
            if url_avatar != None :
                url_avatar = url_avatar.find('span',style = re.compile('background-image: url'))
                if url_avatar != None:
                    temp = re.findall( r'\w+://[\w.]+\w', url_avatar.get('style'))
                    temp.extend(re.findall(r'/+\w+-[\w-]+',url_avatar.get('style')))
                    url_avatar = ''.join(temp) +'.jpg'
                else : url_avatar = 'None'
            else : url_avatar = 'None'
            row += url_avatar + ','
            
            # id_user
            id_user = soup.find('link',{'rel': 'alternate'}, href = re.compile('android-app://com.soundcloud.android/soundcloud/users:'))
            if id_user != None : 
                id_user = re.findall(r'[0-9]+',id_user.get('href'))
                id_user = ''.join(id_user)
            else : id_user  = 'None'
            row += id_user + ','
            
            # verified, username
            verified = 'False'
            username = soup.find('h2', {'class' : "profileHeaderInfo__userName g-type-shrinkwrap-block g-type-shrinkwrap-large-primary sc-text-h1"} ) 
            if username != None: 
                username = username.text.strip()
                list = username.split('\n')
                if len(list) > 1 :
                    username = list[0].strip()
                    username = username.split(',')
                    username =' '.join(username)
                    
                    if list[1].strip() == 'Verified' : verified = 'True'
            else: username ='None'
            row += username + ',' + verified + ',' 
            
            # name
            name = soup.find('h3',{'class' : 'profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary'})
            if name != None: 
                name = name.text.strip()
                name = name.split(',')
                name = ' '.join(name)      
            else : name = 'None'
            row += name + ','
            
            # address
            city = ''
            country = ''
            address = soup.find('h3',{'class': "profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary sc-mt-1x"})
            if address != None:
                address = address.text.strip()
                address = address.split(',')
                city = address[0]
                if len(address) == 1:
                    country = address[0] 
                else :
                    city = address[0]
                    country = ' '.join(address[1:])
            else : city = 'None' ; country ='None'         
            row += city + ',' + country + ','
            
            # description
            description = soup.find('div',{'class' : 'truncatedUserDescription__content'})
            if description != None: 
                description = description.div.text
                description = description.strip().split(',')
                description = ' '.join(description)  
            else : description = 'None'
            row += description + ','
            
            # pro_unlimited ,pro
            pro_unlimited = soup.find('span', {'class':'sc-status-label sc-status-label-pro-u sc-status-label-dark sc-ir'})
            if pro_unlimited != None: 
                pro_unlimited = pro_unlimited.text.strip();
                if pro_unlimited == 'Pro Unlimited': pro_unlimited = 'True'
            else : pro_unlimited = 'False'
            row += pro_unlimited + ','
            pro = soup.find('span', {'class':'sc-status-label sc-status-label-pro sc-status-label-dark sc-ir'})
            if pro != None: 
                pro = pro.text.strip();
                if pro == 'Pro': pro = 'True'
            else : pro = 'False'
            row += pro + ','
            
            # follower
            url_followers = ''
            followers_count = ''
            followers = soup.find('a', {'class' : 'infoStats__statLink sc-link-light sc-link-primary'}, title = re.compile('follower'))
            if followers != None :
                url_followers = url_homepage + followers.get('href')
                followers_count = followers.div.text.strip().split(',')
                followers_count = ''.join(followers_count)
            else : 
                url_followers = 'None' ; followers_count = '0'
            row +=  url_followers + ',' + followers_count + ','
            
            # following
            url_following = ''
            following_count = ''
            following = soup.find('a', {'class' : 'infoStats__statLink sc-link-light sc-link-primary'},title = re.compile('Following'))
            if following != None:
                url_following = url_homepage + following.get('href')
                following_count = following.div.text.strip().split(',')
                following_count = ''.join(following_count)
            else :
                url_following = 'None' ; following_count = '0'
            row += url_following + ',' + following_count + ','
            
            # track
            url_track = ''
            track_count = ''
            track = soup.find('a', {'class' : 'infoStats__statLink sc-link-light sc-link-primary'},title = re.compile('track'))
            if track != None :
                url_track = url_homepage + track.get('href')
                track_count = track.div.text.strip().split(',')
                track_count = ''.join(track_count)
            else: 
                url_track = 'None' ; track_count = '0'
            row += url_track + ',' + track_count + ','
            
            # url_station
            url_station = soup.find('a',{'class' : 'sc-button-startstation sc-button-secondary sc-button sc-button-medium sc-button-responsive'}, title = re.compile('Go to station'))
            if url_station != None :
                url_station = url_homepage + url_station.get('href')
            else : url_station = 'None'
            row += url_station + ','
            
            # like by user
            url_like = ''
            like_count = ''
            like = soup.find('article', {'class' : 'sidebarModule g-all-transitions-200-linear likesModule'})
            if like != None :
                url_like = url_homepage + like.a.get('href')
                like_count = like.find('span',{'class' : 'sidebarHeader__actualTitle sc-text-h3'})
                like_count = re.findall(r'[0-9]',like_count.text.strip())
                like_count = ''.join(like_count)   
            else :
                url_like = 'None' ; like_count = '0'
            row += url_like + ',' + like_count + ','
            
            # url_visual
            url_visual = soup.find('div', {'class' : 'profileHeaderBackground__visual'})
            if url_visual != None : 
                url_visual = url_visual.get('style')
                temp = re.findall( r'\w+://[\w.]+\w',url_visual)
                temp.extend(re.findall(r'/+\w+-[\w-]+',url_visual))
                url_visual = ''.join(temp) +'.jpg'
            else : url_visual =' None'
            row += url_visual + ','
            
            # comment_count
            commented_count = 0
            commented = soup.find('article', {'class' : 'sidebarModule g-all-transitions-200-linear commentsModule'})
            if commented != None :
                commented_count = commented.find('span',{'class' : 'sidebarHeader__actualTitle sc-text-h3'})
                commented_count = re.findall(r'[0-9]',commented_count.text.strip())
                commented_count = ''.join(commented_count)
            else : 
                commented_count = '0'  
            row += commented_count + '\n'
            f_writer.write(row)

>______________________________________________________________________________
> **NOTE**:
>
>     Dưới đây là hàm viết để thu thập thông tin về  playlist , album, repost của user
>
>     Vì thời gian chạy lấy các thông tin này chiếm nhiều thời gian nên em loại bỏ nó. 
>
>     Và trích dẫn để quý thây cô tham khảo
>______________________________________________________________________________


In [116]:
# Dưới đây là hàm viết để thu thập thông tin về playlist ,album, repost của user
def collect_other_attUser(url_user):
    # playlist
    url_playlist = url_user + '/sets'
    driver = webdriver.Chrome("chromedriver.exe", options = options)
    driver.get(url_playlist)
    driver.implicitly_wait(1)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    new_soup = BeautifulSoup(driver.page_source,'lxml')
    driver.close()
    playlist = new_soup.findAll('li',{'class':'soundList__item'})
    playlist_count = len(playlist)
    row += url_playlist + ',' + str(playlist_count) + ','
    # reposts by user
    url_repost = url_user + '/reposts'
    driver = webdriver.Chrome("chromedriver.exe" , options = options)
    driver.get(url_repost)
    driver.implicitly_wait(1)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    new_soup = BeautifulSoup(driver.page_source,'lxml') 
    driver.close()
    repost = new_soup.findAll('li',{'class' : 'soundList__item'})
    repost_count = len(repost)
    row += url_repost + ',' + str(repost_count) + ','
    # albums
    url_album = url_user + '/albums'
    driver = webdriver.Chrome("chromedriver.exe", options = options)
    driver.get(url_album)
    driver.implicitly_wait(1)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    new_soup = BeautifulSoup(driver.page_source,'lxml') 
    driver.close()
    album = new_soup.findAll('li',{'class' : 'soundList__item'})
    album_count = len(album)
    row += url_album + ',' + str(album_count) + ','

In [43]:
# Test
collect_information_users(url_users, correct_user_file)

In [44]:
# check dữ liệu đã thu thập từ trước có đủ 1000 dòng không
list_users = pd.read_csv(correct_user_file,sep =',',encoding ='utf8')
list_users.head(100)

,url_user,url_avatar,id_user,username,verified,name,city,country,description,pro_unlimited,...,followers_count,url_following,followings_count,url_track,tracks_count,url_station,url_like,likes_count,url_visual,comments_count
0,https://soundcloud.com/athackray,https://i1.sndcdn.com/avatars-RyT2YtSSdAWHNNlL...,8490836,abigail,False,Abigail Thackray,None,None,None,False,...,726,https://soundcloud.com/athackray/following,319,https://soundcloud.com/athackray/tracks,27,https://soundcloud.com/discover/sets/artist-st...,https://soundcloud.com/athackray/likes,254,https://i1.sndcdn.com/visuals-000008490836-vPP...,129
1,https://soundcloud.com/absofcourse,https://i1.sndcdn.com/avatars-U6yJsUzSJDMKBsc5...,60503817,Abigail Barlow,False,Abigail Ross Barlow,Los Angeles,United States,21-year-old songstress Abigail Barlow not only...,True,...,4909,https://soundcloud.com/absofcourse/following,56,https://soundcloud.com/absofcourse/tracks,21,https://soundcloud.com/discover/sets/artist-st...,https://soundcloud.com/absofcourse/likes,11,https://i1.sndcdn.com/visuals-000060503817-Skz...,2
2,https://soundcloud.com/user-560957692,None,246730534,Abigail,False,None,None,None,None,False,...,13,https://soundcloud.com/user-560957692/following,0,https://soundcloud.com/user-560957692/tracks,27,https://soundcloud.com/discover/sets/artist-st...,None,0,None,1
3,https://soundcloud.com/user-465703280,https://i1.sndcdn.com/avatars-LrNR2IBk0IknieNR...,982586593,Abigail,False,None,None,None,None,False,...,1,https://soundcloud.com/user-465703280/following,0,https://soundcloud.com/user-465703280/tracks,140,https://soundcloud.com/discover/sets/artist-st...,None,0,None,0
4,https://soundcloud.com/abigailofficial,https://i1.sndcdn.com/avatars-000212544756-brg...,212027482,Abigail,False,None,None,None,None,True,...,40,https://soundcloud.com/abigailofficial/following,0,https://soundcloud.com/abigailofficial/tracks,179,https://soundcloud.com/discover/sets/artist-st...,None,0,None,0
5,https://soundcloud.com/user-9726483,https://i1.sndcdn.com/avatars-JXEUwxJffHKTyuk6...,1010288254,Abigail,False,None,None,None,None,False,...,0,https://soundcloud.com/user-9726483/following,0,https://soundcloud.com/user-9726483/tracks,116,https://soundcloud.com/discover/sets/artist-st...,None,0,https://i1.sndcdn.com/visuals-001010288254-yKV...,0
6,https://soundcloud.com/apikalemusic,https://i1.sndcdn.com/avatars-LghESQXNjLmJhPmM...,478848018,ABIGAIL,False,None,Salt Lake City,United States,GO HARD OR STAY TRAPPEDabigailxartist@gmail.co...,False,...,161,https://soundcloud.com/apikalemusic/following,4,https://soundcloud.com/apikalemusic/tracks,15,https://soundcloud.com/discover/sets/artist-st...,https://soundcloud.com/apikalemusic/likes,2,https://i1.sndcdn.com/visuals-000478848018-tEl...,10
7,https://soundcloud.com/abigailcantika,https://i1.sndcdn.com/avatars-000264126584-rm9...,10542860,AbigailCantika,False,Cantika Abigail,jakarta,Indonesia,None,False,...,9074,https://soundcloud.com/abigailcantika/following,115,https://soundcloud.com/abigailcantika/tracks,31,https://soundcloud.com/discover/sets/artist-st...,https://soundcloud.com/abigailcantika/likes,137,None,6
8,https://soundcloud.com/abigail-lynn-528707031,https://i1.sndcdn.com/avatars-000179352118-lhb...,177139266,Abigail,False,Abigail Lynn,None,None,None,False,...,9,https://soundcloud.com/abigail-lynn-528707031/...,3,https://soundcloud.com/abigail-lynn-528707031/...,0,None,https://soundcloud.com/abigail-lynn-528707031/...,3,None,0
9,https://soundcloud.com/abigail-chew,https://i1.sndcdn.com/avatars-000312074637-drl...,25854654,abigailchew,False,Abigail Chew,Kuala Lumpur,Malaysia,just a girl with big dreams.I self-record at home,False,...,1863,https://soundcloud.com/abigail-chew/following,2,https://soundcloud.com/abigail-chew/tracks,18,https://soundcloud.com/discover/sets/artist-st...,https://soundcloud.com/abigail-chew/likes,5,https://i1.sndcdn.com/visuals-000025854654-A2l...,7


In [39]:
# file user.csv chứa thông tin hơn 1000 user đã thu thập từ trước bằng html-parse.
# check dữ liệu đã thu thập từ trước có đủ 1000 dòng không
# test
correct_file = pd.read_csv(user_file,sep =',',encoding = 'utf8')
assert len(correct_file) > 1000
correct_file.head(100)

,url_user,url_avatar,id_user,username,verified,name,city,country,description,pro_unlimited,...,url_like,likes_count,url_playlist,playlist_count,url_repost,reposts_count,url_album,albums_count,url_visual,comments_count
0,https://soundcloud.com/athackray,https://i1.sndcdn.com/avatars-RyT2YtSSdAWHNNlL...,8490836,abigail,False,Abigail Thackray,None,None,None,False,...,https://soundcloud.com/athackray/likes,254,https://soundcloud.com/athackray/sets,3,https://soundcloud.com/athackray/reposts,1,https://soundcloud.com/athackray/albums,0,https://i1.sndcdn.com/visuals-000008490836-vPP...,129
1,https://soundcloud.com/user-9726483,https://i1.sndcdn.com/avatars-JXEUwxJffHKTyuk6...,1010288254,Abigail,False,None,None,None,None,False,...,None,0,https://soundcloud.com/user-9726483/sets,0,https://soundcloud.com/user-9726483/reposts,0,https://soundcloud.com/user-9726483/albums,0,https://i1.sndcdn.com/visuals-001010288254-yKV...,0
2,https://soundcloud.com/apikalemusic,https://i1.sndcdn.com/avatars-LghESQXNjLmJhPmM...,478848018,ABIGAIL,False,None,Salt Lake City,United States,GO HARD OR STAY TRAPPEDabigailxartist@gmail.co...,False,...,https://soundcloud.com/apikalemusic/likes,2,https://soundcloud.com/apikalemusic/sets,0,https://soundcloud.com/apikalemusic/reposts,5,https://soundcloud.com/apikalemusic/albums,2,https://i1.sndcdn.com/visuals-000478848018-tEl...,10
3,https://soundcloud.com/user-560957692,None,246730534,Abigail,False,None,None,None,None,False,...,None,0,https://soundcloud.com/user-560957692/sets,0,https://soundcloud.com/user-560957692/reposts,0,https://soundcloud.com/user-560957692/albums,0,None,0
4,https://soundcloud.com/abigail-chew,https://i1.sndcdn.com/avatars-000312074637-drl...,25854654,abigailchew,False,Abigail Chew,Kuala Lumpur,Malaysia,just a girl with big dreams.I self-record at home,False,...,https://soundcloud.com/abigail-chew/likes,5,https://soundcloud.com/abigail-chew/sets,1,https://soundcloud.com/abigail-chew/reposts,0,https://soundcloud.com/abigail-chew/albums,0,https://i1.sndcdn.com/visuals-000025854654-A2l...,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://soundcloud.com/georgiy-zhulakashvili,https://i1.sndcdn.com/avatars-F4tFW3GtOPL2TtZr...,361411991,beridze,False,Georgiy Zhulakashvili,None,None,None,False,...,https://soundcloud.com/georgiy-zhulakashvili/l...,70,https://soundcloud.com/georgiy-zhulakashvili/sets,1,https://soundcloud.com/georgiy-zhulakashvili/r...,2,https://soundcloud.com/georgiy-zhulakashvili/a...,0,None,5
96,https://soundcloud.com/avamompreneur,https://i1.sndcdn.com/avatars-000715322077-ki0...,490084287,AVA,False,Ava Chitsuthipakorn,Nakhonsawan,Thailand,Podcast ของคุณแม่ที่มีความสุขกับความเป็นแม่เพื...,False,...,None,0,https://soundcloud.com/avamompreneur/sets,0,https://soundcloud.com/avamompreneur/reposts,0,https://soundcloud.com/avamompreneur/albums,0,https://i1.sndcdn.com/visuals-000490084287-a1p...,1
97,https://soundcloud.com/yes-andres,https://i1.sndcdn.com/avatars-000698802964-tn4...,628731855,Alias,False,None,None,None,None,False,...,https://soundcloud.com/yes-andres/likes,4,https://soundcloud.com/yes-andres/sets,3,https://soundcloud.com/yes-andres/reposts,0,https://soundcloud.com/yes-andres/albums,0,None,0
98,https://soundcloud.com/berkay-tasbas,https://i1.sndcdn.com/avatars-000597417357-b8o...,605164551,bosu,False,Berkay Tasbas,None,None,None,False,...,https://soundcloud.com/berkay-tasbas/likes,2,https://soundcloud.com/berkay-tasbas/sets,3,https://soundcloud.com/berkay-tasbas/reposts,0,https://soundcloud.com/berkay-tasbas/albums,0,None,1


# THU THẬP DỮ LIỆU CHO PLAYLIST

---
# Setup các biến toàn cục

Ở bước này sẽ setup các biến như:
- `sleep_time`. Dùng để đợi webdrive load đủ số lượng link cần lấy
- `cache`
- `url`. Url mẫu cho việc search playlists trên [SoundCloud](https://soundcloud.com/search/sets?q=the). 
- `key_playlist_file` . đường dẫn đến file chứa key cho việc search playlist
- `correct_playlist_file` . đường dẫn đến file `correct_playlist.csv` mà chương trình vừa mới tạo ra 
- `playlist_file` . đường dẫn đến file `playlist.csv` mà chương trình vừa mới tạo ra mà bọn em đã thu thập được

In [5]:
sleep_time = 1

In [6]:
requests_cache.install_cache("cache", backend="sqlite", expire_after= - 1)

In [7]:
url = "https://soundcloud.com/search/sets?q="

In [8]:
key_playlist_file  = "./key_search/key_playlist.txt"
correct_playlist_file = "./correct_file/correct_playlist.csv"
playlist_file = "./HTML_data/playlist.csv"

---
# Các thao tác xử lí

Có 3 thao tác xử lí chính trong phần bài làm này:
- Đọc 100 từ khóa từ khóa
- Search lấy một số lượng *link* phù hợp bằng `Selenium`
- Phân tích các trường thông tin đối với mỗi list bằng `Requests` và `BeautifulSoup`
- Ghi dữ liệu ra file `playlist.csv`

## Đọc 100 từ khóa chính
File `key_playlist.txt` chứa 100 từ khóa phổ biến.

In [9]:
file = open(key_playlist_file,"r")
name = file.readlines()
for key_search in range(len(name)):
    name[key_search] = name[key_search].rstrip()
file.close()

In [10]:
#Test 
for i in range(10):
      print(str(i + 1) + ".", name[i]) 
print (".\n.\n.\n.")
print(str(len(name)) + ".", name[-1])

1. the
2. at
3. there
4. some
5. my
6. of
7. be
8. use
9. her
10. than
.
.
.
.
100. via


## Search lấy một số lượng *link* phù hợp bằng `Selenium`

Ở bước này, có ba thứ tự hoạt động như sau:
1. Khai báo các biến cần thiết
2. Khai báo các hàm xử lí.
3. Phần xử lí chính.

### 1.Khai báo các biến cần thiết

- `list_link` là `list` chứa các link dẫn đến các `playplist`
- `range_name` cho biết số lượng tab trên một cửa sổ `webdrive`
- `num_link_need` là số lượng link cần láy trên môi tab search

In [11]:
list_link = []
range_name = 20
num_link_need = 12

### 2.Khai báo các hàm xử lí.

Hàm `extractLink` dùng để lấy đường link từ dữ phần tag đã có. 

Vd: ~~`<a class="sc-link-primary soundTitle__title sc-link-dark sc-text-h4 href="/`~~ davilleofficial/sets/daville-on-my-mind-2007 ~~`">
<span class="">Da'Ville - On My Mind (2007)</span>
</a>`~~

In [12]:
def extractLink(listt):
    for i in range(len(listt)):
        listt[i] = "https://soundcloud.com" + listt[i]['href']

### 3.Xử lí chính

In [13]:
for i in range(int((len(name)/range_name))):
    #Search với 20 tab mỗi tab search ứng với một từ khóa. Do đó sẽ có 5 vòng lặp for
    dr = webdriver.Chrome(kind_chromedrive)
    dr.get("https://soundcloud.com")
    window_first = dr.window_handles[0] #Lấy window_hand cho tab đầu tiên. Vì sau khi search xong 20 tab thì sẽ xóa tab này.

    for j in range(range_name):
        dr.execute_script("window.open('{}');".format("https://soundcloud.com/search/sets?q=" +name[i*20 + j]))

    #Quay lại tab đầu tiên và đóng nó.    
    dr.switch_to_window(window_first)
    dr.close()
    
    #Làm việc với 20 tab 
    for j in range(range_name):
        dr.switch_to.window(dr.window_handles[range_name - j - 1])
        
        #Khi số lượng link playlist lớn hơn 12 thì sẽ lấy 12 link đầu tiên và thoát.
        #Nếu page chưa load đủ thì đợi một giây sau và lấy lại số link cần.
        time_count = 0
        while(True):
            dr.execute_script("window.scrollTo(0, document.body.scrollHeight)") 
            html = dr.page_source
            soup = BeautifulSoup(html, "html.parser")
            if(len(soup.find_all("a", class_="sc-link-primary soundTitle__title sc-link-dark sc-text-h4")) > num_link_need):
                list_link.extend(soup.find_all("a", class_="sc-link-primary soundTitle__title sc-link-dark sc-text-h4")[0:num_link_need])
                break
            time.sleep(sleep_time)
            time_count = time_count + 1
            if (time_count == 7):
                dr.refresh()
                time_count = 0
        dr.close()
        
#Lấy link từ các tag thô        
extractLink(list_link)

#Loại bỏ trùng lặp
list_link = list(dict.fromkeys(list_link))

/tmp/ipykernel_42652/2990732577.py:11: DeprecationWarning: use driver.switch_to.window instead
  dr.switch_to_window(window_first)


In [14]:
#Test
print("Độ dài list link:", len(list_link))

for i in range(10):
      print(str(i + 1) + ".", list_link[i]) 
print (".\n.\n.\n.")
print(str(len(list_link)) + ".", list_link[-1])

Độ dài list link: 1171
1. https://soundcloud.com/davilleofficial/sets/daville-on-my-mind-2007
2. https://soundcloud.com/davilleofficial/sets/daville-in-heaven-2005
3. https://soundcloud.com/davilleofficial/sets/daville-krazy-love-2012
4. https://soundcloud.com/davilleofficial/sets/daville-2013
5. https://soundcloud.com/davilleofficial/sets/daville-favorites
6. https://soundcloud.com/ed-coburn/sets/popesque
7. https://soundcloud.com/akash-pande-182043212/sets/maurice-west-the-best-trance-x
8. https://soundcloud.com/ed-cashman/sets/the-long-playlist-by-me
9. https://soundcloud.com/user-439673996/sets/the
10. https://soundcloud.com/monopiano1984/sets/wow
.
.
.
.
1171. https://soundcloud.com/user-344309980/sets/via


## Phân tích các trường thông tin đối với mỗi list bằng `Requests` và `BeautifulSoup`

Ở bước này, có ba thứ tự hoạt động như sau:
1. Khai báo các biến cần thiết
2. Khai báo các hàm xử lí.
3. Phần xử lí chính.

### 1. Khai báo các biến cần thiết


Khai báo các trường thông tin sẽ lấy đối với dữ liệu của **playlist**
1. `artwork_urls` : link của artwork
2. `created_ats` : địa chỉ tạo artwork
3. `descriptions` : phần giới thiệu
4. `durations` : thời lượng phát của list (milisecond)
5. `embeddable_bys` : Kiểu cho phép nhúng
6. `genres` : thể loại
7. `ids` : list id
8. `kinds` : loại dữ liệu (playlist)
9. `label_names` : tên nhãn
10. `last_modifieds` : ngày chỉnh sửa cuối
11. `licenses` : loại bản quyền
12. `likes_counts` : số lượng thích
13. `managed_by_feedss` : có được quản lý bởi nguồn dữ liệu nào không
14. `permalinks` : đường dẫn đến playlist
15. `permalink_urls` : url đến playlist
16. `publics` : có được public không
17. `purchase_titles` : tiêu đề về việc mua bán
18. `purchase_urls` : link mua bán
19. `release_dates` : ngày phát hành
20. `reposts_counts` : số lượng repost
21. `secret_tokens` : token của playlist
22. `sharings` : chế độ chia sẻ
23. `tag_lists` : danh sách các thẻ
24. `titles` : tiêu dề
25. `uris` : link api của playlist
26. `user_ids` : id của user chủ playlist
27. `set_types` : một set các kiểu
28. `is_albums` : có phải album không
29. `published_ats` : ngày công khai
30. `display_dates` : hiển thị ngày phát hành
31. `users` : thông tin user chủ playlist
32. `trackss` : thông tin các track
33. `track_counts`: số lượng các track
44. `user_names` : tên user chủ playlist
55. `tracksIdss` : chuỗi các trackId mà playlist này chứa

In [15]:
artwork_urls = []
created_ats = []
descriptions = []
durations = []
embeddable_bys = []
genres = []
ids = []
kinds = []
label_names = []
last_modifieds = []
licenses = []
likes_counts = []
managed_by_feedss = []
permalinks = []
permalink_urls = []
publics = []
purchase_titles = []
purchase_urls = []
release_dates = []
reposts_counts = []
secret_tokens = []
sharings = []
tag_lists = []
titles = []
uris = []
user_ids = []
set_types = []
is_albums = []
published_ats = []
display_dates = []
users = []
trackss = []
track_counts = []
user_names = []
tracksIdss = []

### 2. Khai báo các hàm xử lí

Hàm `getHydration` dùng để lấy được phần tử script có chứa `window.__sc_hydration`

In [16]:
def getHydration(head_data):
    hydration = ""
    for i in range(len(head_data)):
        if ("window.__sc_hydration" in str(head_data[i].extract())):
            hydration = str(head_data[i].extract)
            a = re.findall("= (.+);",hydration)
            b = json.loads(a[0])
            info_repo = b[-1]["data"]
    return info_repo

Hàm `extract_TrackIds` dùng để lấy được số `track_id` trong `tracks`

In [17]:
def extract_TrackIds(tracks):
    trackIds = ""
    for i in range(len(tracks)):
        trackIds = trackIds + str(tracks[i]["id"]) + ","
    return trackIds

### 3. Xử lí chính

In [18]:
for i in range(len(list_link)):

    #request và soup
    respond = requests.get(list_link[i])

    if(not respond.from_cache):
        #Tránh bị chặn bởi trang web
        time.sleep(random.randrange(sleep_time + 1))
        
    soup = BeautifulSoup(respond.text.encode('iso-8859-1').decode('utf-8'),"html.parser")

    #Lấy tất cả script có trong pagesource
    head_data = soup.find_all("script")
    
    #Lấy script có chứa window__.cs_hydration 
    info_repo = getHydration(head_data)
    
    #Phân tích

    artwork_urls.append(info_repo["artwork_url"])
    
    created_ats.append(info_repo["artwork_url"])
    
    descriptions.append(info_repo["description"])
    
    durations.append(info_repo["duration"])
    
    embeddable_bys.append(info_repo["embeddable_by"])
    
    genres.append(info_repo["genre"])
    
    ids.append(info_repo["id"])
    
    kinds.append(info_repo["kind"])
    
    label_names.append(info_repo["label_name"])
    
    last_modifieds.append(info_repo["last_modified"])
    
    licenses.append(info_repo["license"])
    
    likes_counts.append(info_repo["likes_count"])
    
    managed_by_feedss.append(info_repo["managed_by_feeds"])
    
    permalinks.append(info_repo["permalink"])
    
    permalink_urls.append(info_repo["permalink_url"])
    
    publics.append(info_repo["public"])
    
    purchase_titles.append(info_repo["purchase_title"])
    
    purchase_urls.append(info_repo["purchase_url"])
    
    release_dates.append(info_repo["release_date"])
    
    reposts_counts.append(info_repo["reposts_count"])
    
    secret_tokens.append(info_repo["secret_token"])
    
    sharings.append(info_repo["sharing"])
    
    tag_lists.append(info_repo["tag_list"])
    
    titles.append(info_repo["title"])
    
    uris.append(info_repo["uri"])
    
    user_ids.append(info_repo["user_id"])
    
    set_types.append(info_repo["set_type"])
    
    is_albums.append(info_repo["is_album"])
    
    published_ats.append(info_repo["published_at"])
    
    display_dates.append(info_repo["display_date"])
    
    users.append(str(info_repo["user"]))
    
    trackss.append(str(info_repo["tracks"]))
    
    track_counts.append(str(info_repo["track_count"]))
    
    user_names.append(info_repo["user"]["username"])
    
    tracksIdss.append(extract_TrackIds(info_repo["tracks"]))
    

## Ghi dữ liệu ra file `playlist.csv`

In [19]:
repo ={
    'artwork_url' : artwork_urls,
    'created_at' : created_ats,
    'descriptions' : descriptions,
    'duration' : durations,
    'embeddable_by' : embeddable_bys,
    'genre' : genres,
    'id' : ids,
    'kind' : kinds,
    'label_name' : label_names,
    'last_modified': last_modifieds,
    'license' : licenses,
    'likes_count' : likes_counts,
    'managed_by_feeds' : managed_by_feedss,
    'permalink' : permalinks,
    'permalink_url' : permalink_urls,
    'public' : publics,
    'purchase_title' : purchase_titles,
    'purchase_url' : purchase_urls,
    'release_date' : release_dates,
    'reposts_count' : reposts_counts,
    'secret_token' : secret_tokens,
    'sharing' : sharings,
    'tag_list' : tag_lists,
    'title' : titles,
    'uri' : uris,
    'user_id' : user_ids,
    'set_type' : set_types,
    'is_album' : is_albums,
    'published_at' : published_ats,
    'display_date' : display_dates,
    'user' : users,
    'tracks' : trackss,
    'track_count' : track_counts,
    'user_name' : user_names,
    'tracksIds' : tracksIdss
}

df = pd.DataFrame(data=repo)
df.to_csv(correct_playlist_file, sep = ',', index=False)

Xem cấu trúc file `playlist.csv` mà bọn em nộp như thế nào và kiểm tra xem file đã hơn 1000 > `row` hay chưa

In [21]:
#Test
df = pd.read_csv(playlist_file, sep=',')
assert len(df) > 1000
df

,artwork_url,created_at,descriptions,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,user_id,set_type,is_album,published_at,display_date,user,tracks,track_count,user_name,tracksIds
0,https://i1.sndcdn.com/artworks-000043495227-g4...,https://i1.sndcdn.com/artworks-000043495227-g4...,davilleofficial@gmail.com\r\nfacebook.com/davi...,8459153,all,Reggae,4185074,playlist,VP Records,2019-08-31T22:20:40Z,...,39236595,NaN,False,NaN,2013-03-21T11:17:26Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,39,Da'Ville,"84245530,84245532,84245533,84245534,84245535,8..."
1,https://i1.sndcdn.com/artworks-000043491308-89...,https://i1.sndcdn.com/artworks-000043491308-89...,davilleofficial@gmail.com\r\nfacebook.com/davi...,8679045,all,Reggae,4184221,playlist,Fashozy Records,2019-08-31T22:20:41Z,...,39236595,NaN,False,NaN,2013-03-21T09:30:40Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,40,Da'Ville,"84237972,84237967,84237965,84237960,84237977,8..."
2,https://i1.sndcdn.com/artworks-000043503850-3a...,https://i1.sndcdn.com/artworks-000043503850-3a...,davilleofficial@gmail.com\r\nfacebook.com/davi...,5441763,all,Reggae,4187276,playlist,Fashozy Records,2019-08-31T22:20:40Z,...,39236595,NaN,False,NaN,2013-03-21T14:03:16Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,26,Da'Ville,"84260152,84260154,84260155,84260156,84260157,8..."
3,NaN,NaN,NaN,5231590,all,NaN,14787366,playlist,NaN,2019-08-31T22:20:39Z,...,39236595,NaN,False,NaN,2013-11-16T20:02:49Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,25,Da'Ville,"120457313,453844149,452028996,452028414,450957..."
4,NaN,NaN,NaN,1896056,all,NaN,859147910,playlist,NaN,2019-08-31T22:20:34Z,...,39236595,NaN,False,2019-08-31T22:08:13Z,2019-08-31T22:08:13Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,9,Da'Ville,"673826063,450955557,453844149,450957921,120457..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,NaN,NaN,NaN,34041129,all,NaN,429139178,playlist,NaN,2019-10-07T01:05:21Z,...,387037697,NaN,False,2018-01-22T09:27:30Z,2018-01-22T09:27:30Z,{'avatar_url': 'https://a1.sndcdn.com/images/d...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,80,eryanty,"233161566,330604163,295414933,71563123,9363459..."
1171,NaN,NaN,NaN,45877677,all,NaN,96284674,playlist,NaN,2015-04-07T12:34:08Z,...,146324826,NaN,False,NaN,2015-04-07T12:34:08Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,154,user542017728,"158721013,162777996,181404648,97939459,1125726..."
1172,NaN,NaN,NaN,57959438,all,NaN,106307249,playlist,NaN,2020-11-17T22:15:22Z,...,134544530,NaN,False,NaN,2015-05-10T15:19:45Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",175,Mathrus Ferreira,"181549500,230305086,108032066,72785037,8310814..."
1173,NaN,NaN,NaN,44536291,all,NaN,1336376044,playlist,NaN,2021-10-25T09:24:16Z,...,1044283210,NaN,False,2021-10-24T21:47:57Z,2021-10-24T21:47:57Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,32,Dewi Natasya,"1113242530,967413454,971380666,1126703938,1117..."


In [23]:
#Test
df = pd.read_csv(correct_playlist_file, sep=',')
assert len(repo) == len(df.columns)
assert len(df) > 1000
print(len(list_link))
df

1171


,artwork_url,created_at,descriptions,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,user_id,set_type,is_album,published_at,display_date,user,tracks,track_count,user_name,tracksIds
0,https://i1.sndcdn.com/artworks-000043495227-g4...,https://i1.sndcdn.com/artworks-000043495227-g4...,davilleofficial@gmail.com\r\nfacebook.com/davi...,8459153,all,Reggae,4185074,playlist,VP Records,2019-08-31T22:20:40Z,...,39236595,NaN,False,NaN,2013-03-21T11:17:26Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,39,Da'Ville,"84245530,84245532,84245533,84245534,84245535,8..."
1,https://i1.sndcdn.com/artworks-000043491308-89...,https://i1.sndcdn.com/artworks-000043491308-89...,davilleofficial@gmail.com\r\nfacebook.com/davi...,8679045,all,Reggae,4184221,playlist,Fashozy Records,2019-08-31T22:20:41Z,...,39236595,NaN,False,NaN,2013-03-21T09:30:40Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,40,Da'Ville,"84237972,84237967,84237965,84237960,84237977,8..."
2,https://i1.sndcdn.com/artworks-000043503850-3a...,https://i1.sndcdn.com/artworks-000043503850-3a...,davilleofficial@gmail.com\r\nfacebook.com/davi...,5441763,all,Reggae,4187276,playlist,Fashozy Records,2019-08-31T22:20:40Z,...,39236595,NaN,False,NaN,2013-03-21T14:03:16Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,26,Da'Ville,"84260152,84260154,84260155,84260156,84260157,8..."
3,NaN,NaN,NaN,5231590,all,NaN,14787366,playlist,NaN,2019-08-31T22:20:39Z,...,39236595,NaN,False,NaN,2013-11-16T20:02:49Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,25,Da'Ville,"120457313,453844149,452028996,452028414,450957..."
4,NaN,NaN,NaN,1896056,all,NaN,859147910,playlist,NaN,2019-08-31T22:20:34Z,...,39236595,NaN,False,2019-08-31T22:08:13Z,2019-08-31T22:08:13Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,9,Da'Ville,"673826063,450955557,453844149,450957921,120457..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,NaN,NaN,NaN,34041129,all,NaN,429139178,playlist,NaN,2019-10-07T01:05:21Z,...,387037697,NaN,False,2018-01-22T09:27:30Z,2018-01-22T09:27:30Z,{'avatar_url': 'https://a1.sndcdn.com/images/d...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,80,eryanty,"233161566,330604163,295414933,71563123,9363459..."
1167,NaN,NaN,NaN,45877677,all,NaN,96284674,playlist,NaN,2015-04-07T12:34:08Z,...,146324826,NaN,False,NaN,2015-04-07T12:34:08Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,154,user542017728,"158721013,162777996,181404648,97939459,1125726..."
1168,NaN,NaN,NaN,57959438,all,NaN,106307249,playlist,NaN,2020-11-17T22:15:22Z,...,134544530,NaN,False,NaN,2015-05-10T15:19:45Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,"[{'artwork_url': None, 'caption': None, 'comme...",175,Mathrus Ferreira,"181549500,230305086,108032066,72785037,8310814..."
1169,NaN,NaN,NaN,44536291,all,NaN,1336376044,playlist,NaN,2021-10-25T09:24:16Z,...,1044283210,NaN,False,2021-10-24T21:47:57Z,2021-10-24T21:47:57Z,{'avatar_url': 'https://i1.sndcdn.com/avatars-...,[{'artwork_url': 'https://i1.sndcdn.com/artwor...,32,Dewi Natasya,"1113242530,967413454,971380666,1126703938,1117..."


# THU THẬP DỮ LIỆU CHO TRACKS. NGƯỜI LÀM [BẢO TÍN]

---
# Setup toàn cục

Ở bước này sẽ setup các biến như:
- `sleep_time`. Dùng để đợi webdrive load đủ số lượng link cần lấy
- `link_base` . Url dẫn đến trang chủ của SoundCLoud
- `base_url`. Url mẫu cho việc search playlists trên [SoundCloud](https://soundcloud.com/search/sets?q=the). 
- `correct_track_file` . đường dẫn đến file `correct_track.csv`mà chương trình vừa mới tạo ra
- `track_file` . đường dẫn đến file `track.csv` mà bọn em đã thu thập được
- các `list` thuộc tính cần lấy.

In [2]:
time_sleep = 3 # Đặt thời gian sleep mặc định là 3

In [3]:
link_base = 'https://soundcloud.com' 
base_url = 'https://soundcloud.com/search/sounds?q='

In [4]:
correct_track_file = "./correct_file/correct_track.csv"
track_file = "./HTML_data/track.csv"

**Thông tin các thuộc tính**
1. artwork_url : link file ảnh của track
2. caption : Đầu đề
3. track_url : Link của track
4. comment_count : Số lượng bình luận
5. created_at : Thời gian tạo ra
6. description : Mô tả của chủ sỡ hữu
7. download_count : Số lượng đã tải về
8. duration : Thời lượng
9. genre : Thể loại(rap,pop,...)
10. id : Id của track
11. kind : Loại (track,playlits,user,..)
12. last_modified : Lần cuối được mở
13. license : Giấy phép
14. likes_count : Số lượng thích
15. public : Chế độ public hay không
16. release_date : Ngày đăng
17. reposts_count : Số lượng reposts
18. sharing : Chế độ share (public,private,..)
19. title : Tên track
20. track_format : Loại của track (single,couple,...)
21. user_id : Id của chủ sỡ hữu
22. display_date : Ngày mở
23. track_authorization : Ủy quyền của track
24. policy : Quyền đối với track

In [5]:
artwork_urls = []
captions = []
comment_counts = []
track_urls = []
created_ats = []
descriptions = []
download_counts = []
durations = []
genres = []
ids = []
kinds = []
last_modifieds = []
licenses = []
likes_counts = []
publics = []
release_dates = []
reposts_counts = []
sharings = []
titles = []
track_formats = []
user_ids = []
display_dates = []
track_authorizations = []
policies = []

# Các xử lí chính

In [8]:
browser = webdriver.Chrome(kind_chromedrive) # Mở 1 trình duyệt Chrome

Hàm `getDatas` lấy thông tin với mỗi link dẫn tới track.

In [9]:
def getDatas(url,artwork_urls,captions,track_urls,comment_counts,created_ats,descriptions,download_counts,durations,
              genres,ids,kinds,last_modifieds,licenses,likes_counts,publics,release_dates,repost_counts,sharings,
              titles,track_formats,user_ids,display_dates,track_authorizations,policies):
    browser.get(url)   #Hàm lấy thông tin với mỗi link dẫn tới track
    if not browser:
        return
    
    time.sleep(time_sleep)
    
    track_urls.append(url)
    html_source = browser.page_source  #lấy source của page bằng selenium
    soup = BeautifulSoup(html_source, 'html.parser')   #Tạo ra 1 beautyful soup object
    t = soup.find_all('script')  #Lấy tất cả thẻ script
    
    s = ''
    for sc in t:
        if('window.__sc_hydration' in str(sc)[:31]):   #Kiểm tra thẻ mà trong đó có tồn tại 'window.__sc_hydration'
            st = str(sc)[:-10]  #Loại bỏ các kí tự thừa
            st = st[31:]
            s = json.loads(st)  #Đưa chuỗi có được về kiểu json
  
            if(s[6]['data'] == {}):
                break
            
            #Trích suất dữ liệu từ trong thẻ lấy được
            artwork_urls.append(s[6]['data']['artwork_url'])
            
            captions.append(s[6]['data']['caption'])
            
            comment_counts.append(s[6]['data']['comment_count'])
            
            created_ats.append(s[6]['data']['created_at'])
            
            descriptions.append(s[6]['data']['description'])
            
            download_counts.append(s[6]['data']['download_count'])
            
            durations.append(s[6]['data']['duration'])
            
            genres.append(s[6]['data']['genre'])
            
            ids.append(s[6]['data']['id'])
            
            kinds.append(s[6]['data']['kind'])
            
            last_modifieds.append(s[6]['data']['last_modified'])
            
            licenses.append(s[6]['data']['license'])
            
            likes_counts.append(s[6]['data']['likes_count'])
            
            publics.append(s[6]['data']['public'])
            
            release_dates.append(s[6]['data']['release_date'])
            
            repost_counts.append(s[6]['data']['reposts_count'])
            
            sharings.append(s[6]['data']['sharing'])
            
            titles.append(s[6]['data']['title'])
            
            track_formats.append(s[6]['data']['track_format'])
            
            user_ids.append(s[6]['data']['user_id'])
            
            display_dates.append(s[6]['data']['display_date'])
            
            track_authorizations.append(s[6]['data']['track_authorization'])
            
            policies.append(s[6]['data']['policy'])

### Sử dụng các từ khóa a,b,c,d,..z + d,e,f,g làm hậu tố của link để tạo ra file với hơn 1000 dòng dữ liệu.

In [10]:
count = 0  # Biến tạo ra để lấy khoảng 1000 dòng dữ liệu
check = True #Biến hỗ trợ cho biến count

In [13]:
for one in range(97,123):       # tạo ra các keywords mỗi keyword ra được 10 link nên lấy tổ hợp 25 chữ cái (a->z)
    for two in range(100,105):  #và 4 chữ bất kì để được 100 dòng được 1000 dòng dữ liệu
        item = base_url + chr(one) + chr(two) # Ghép lại để được page chứa danh sách các track với keyword là one+two
        
        browser.get(item)
        if not browser:
               break
                
        time.sleep(time_sleep)
        
        html_source = browser.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        for id in soup.findAll("a", {"class" : "sound__coverArt"}):
            url_suff = id.get('href')  # Tạo vòng lặp để lấy tất cả các đuôi link 
            url = link_base + url_suff # Ghép lại để có được link hoàn chỉnh

            GetDatas(url,artwork_urls,captions,track_urls,comment_counts,created_ats,descriptions,download_counts,durations,
                      genres,ids,kinds,last_modifieds,licenses,likes_counts,publics,release_dates,reposts_counts,
                      sharings,titles,track_formats,user_ids,display_dates,track_authorizations,policies)
            count += 1
        if count > 1150:
            check = False
            break
    if check == False:
        break

### Ghi kết quả ra file

In [15]:
tracks_dataframe = pd.DataFrame({'artwork_url' : artwork_urls, 'caption' : captions, 'track_url' : track_urls, 'comment_count' : comment_counts, 
                                 'created_at' : created_ats,'description' : descriptions,'download_count' : download_counts,
                                 'duration' : durations, 'genre' : genres, 'id' : ids,'kind' : kinds,
                                 'last_modified' : last_modifieds, 'license' : licenses, 'likes_count' : likes_counts, 
                                 'public' : publics, 'release_date' : release_dates, 'repost_count' : reposts_counts, 
                                 'sharing' : sharings, 'title' : titles, 'track_format' : track_formats, 'user_id' : user_ids,
                                 'display_date' : display_dates,'track_authorization' : track_authorizations, 'policy' : policies})
                                 
tracks_dataframe = tracks_dataframe.drop_duplicates('track_url') # Loại bỏ các dòng trùng
tracks_dataframe.to_csv(correct_track_file, index=False, header=True)

Kiểm tra cấu trúc của file `track.csv` mà bọn em đã nộp có lớn hơn 1000 `row` hay chưa

In [94]:
#Test
df = pd.read_csv(track_file, sep=',')
assert len(df) > 1000
df.head(10)

,artwork_url,caption,comment_count,created_at,description,download_count,duration,genre,id,kind,...,public,release_date,repost_count,sharing,title,track_format,user_id,display_date,track_authorization,policy
0,https://i1.sndcdn.com/artworks-000028230067-un...,NaN,2413.0,2012-06-22T19:26:51Z,"Listen to Miguel's newest track ""A D O R N "".",0.0,196383,R&b,50548577,track,...,True,NaN,27216,public,A D O R N,single-track,19058064,2012-06-22T19:26:51Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
1,https://i1.sndcdn.com/artworks-000102578929-xn...,NaN,1580.0,2015-01-07T17:37:29Z,NaN,0.0,268119,love,184922621,track,...,True,NaN,3912,public,New Hindi Love VALENTINE Sad Mashup 2015 La...,single-track,131414429,2015-01-07T17:37:29Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
2,https://i1.sndcdn.com/artworks-000082263302-9p...,NaN,1325.0,2014-06-12T18:05:26Z,زهير البهاوي,100.0,275330,bebo,154070985,track,...,True,NaN,9528,public,بغيت وياما حسيت,single-track,87540633,2014-06-12T18:05:26Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
3,https://i1.sndcdn.com/artworks-NekPFrjguPMF-0-...,NaN,0.0,2021-09-21T14:30:03Z,NaN,0.0,30000,Pop,1128574825,track,...,True,2021-09-14T00:00:00Z,2,public,Mesh Ad el Hawa,single-track,530208174,2021-09-14T00:00:00Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,SNIP
4,https://i1.sndcdn.com/artworks-000379860066-zf...,NaN,839.0,2018-07-26T21:24:37Z,NaN,0.0,237318,NaN,477176841,track,...,True,NaN,3717,public,Dah Lw Etsab (Crazy AD),single-track,206884233,2018-07-26T21:24:37Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
5,https://i1.sndcdn.com/artworks-000113598696-7i...,NaN,270.0,2015-04-16T16:28:34Z,Et7ad El-Kama - Sha3by W T3amel - 2015 _ مهرجا...,0.0,248108,Et7ad,201105387,track,...,True,NaN,3587,public,Et7ad El-Kama - Sha3by W T3amel - 2015 _ مهرجا...,single-track,96735988,2015-04-16T16:28:34Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
6,https://i1.sndcdn.com/artworks-000054440766-3n...,NaN,997.0,2013-07-29T01:52:17Z,Shine bright like a diamond\r\nShine bright li...,0.0,140550,Rihana,103033049,track,...,True,NaN,1846,public,Shine Bright Like A Diamond - Gree AirConditio...,single-track,44235309,2013-07-29T01:52:17Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
7,https://i1.sndcdn.com/artworks-AvaHKybzezR3kqX...,NaN,625.0,2018-11-14T14:43:52Z,NaN,0.0,217723,NaN,529798413,track,...,True,NaN,1636,public,فهد العارف - اهوا خاسرني,single-track,144664052,2018-11-14T14:43:52Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
8,NaN,NaN,521.0,2013-12-11T06:50:47Z,NaN,0.0,314099,Country,124280412,track,...,True,NaN,1455,public,Kuch Iss Trhan,single-track,46512734,2013-12-11T06:50:47Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
9,https://i1.sndcdn.com/artworks-000140405086-bq...,NaN,130.0,2015-12-22T02:43:22Z,باقى الالبوم :\nsoundcloud.com/fifa-music-1/se...,0.0,434169,مهرجان,238605566,track,...,True,NaN,2547,public,الدخلاوية - صاحبي اللى صاحبته - 2016,single-track,96735988,2015-12-22T02:43:22Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW


Kiểm tra thông tin file `correct_track.csv` mới được tạo ra

In [18]:
tracks = pd.read_csv(correct_track_file)
tracks
assert len(tracks) > 1000

,artwork_url,caption,track_url,comment_count,created_at,description,download_count,duration,genre,id,...,public,release_date,repost_count,sharing,title,track_format,user_id,display_date,track_authorization,policy
0,https://i1.sndcdn.com/artworks-000214631455-x0...,NaN,https://soundcloud.com/enzoupload/acustico-1ki...,1737,2017-03-25T19:16:44Z,Acústico 1Kilo - Deixe Me Ir (Baviera Knust E...,100,227600,NaN,314441421,...,True,NaN,10937,public,Acústico 1Kilo - Deixe Me Ir (DOWNLOAD FREE)(...,single-track,189545338,2017-03-25T19:16:44Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
1,https://i1.sndcdn.com/artworks-000185320540-60...,NaN,https://soundcloud.com/mixfeed/alok-sevenn-byob,8176,2015-12-22T21:28:07Z,https://soundcloud.com/livealok &amp; https://...,0,259732,Deep House,238718260,...,True,NaN,120557,public,Alok & Sevenn - BYOB [ FREE DOWNLOAD ],single-track,11180360,2015-12-22T21:28:07Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
2,https://i1.sndcdn.com/artworks-000033695413-9m...,NaN,https://soundcloud.com/yara-yussef/adele-set-f...,4113,2012-11-07T00:52:12Z,NaN,0,216051,NaN,66424164,...,True,NaN,17205,public,Adele | Set Fire To The Rain ( Reggae Version ...,single-track,17365207,2012-11-07T00:52:12Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
3,https://i1.sndcdn.com/artworks-000052261541-7l...,NaN,https://soundcloud.com/nightcore-3/angel-with-...,11912,2013-07-05T22:11:34Z,NaN,100,192983,Nightcore,99845216,...,True,NaN,14407,public,Angel With A Shotgun,single-track,31109873,2013-07-05T22:11:34Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
4,https://i1.sndcdn.com/artworks-000026618899-2s...,NaN,https://soundcloud.com/easymoneydom/asap-rocky...,1940,2012-07-13T18:55:29Z,NaN,0,205949,NaN,52751048,...,True,NaN,48565,public,ASAP Rocky Ft. Lana Del Rey - Ridin',single-track,8852761,2012-07-13T18:55:29Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
5,https://i1.sndcdn.com/artworks-000048548811-ig...,NaN,https://soundcloud.com/iamjfernandez/animals-m...,7974,2013-05-20T23:38:26Z,*Second drop* Animals - Martin Garrix (Trenset...,733578,213440,House/Trap,93130136,...,True,NaN,32763,public,Animals - Martin Garrix VS Trendsetter (J.Fern...,single-track,540952,2013-05-20T23:38:26Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
6,https://i1.sndcdn.com/artworks-000105630275-il...,NaN,https://soundcloud.com/jonah_lieberman/andy-gr...,1834,2015-02-04T20:47:29Z,NaN,0,192697,honey,189505104,...,True,NaN,12647,public,Andy Grammer - Honey I'm Good,single-track,68212122,2015-02-04T20:47:29Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
7,https://i1.sndcdn.com/artworks-000117783655-r4...,NaN,https://soundcloud.com/thehrchy/asap-rocky-lsd,1456,2015-05-23T16:09:18Z,NaN,0,264043,asap,206861316,...,True,NaN,27099,public,ASAP Rocky – LSD,single-track,90825763,2015-05-23T16:09:18Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
8,https://i1.sndcdn.com/artworks-000126467201-w6...,NaN,https://soundcloud.com/espacio-urbano-music/al...,1474,2015-08-15T18:30:17Z,Sigue En Facebook : https://www.facebook.com/p...,0,477798,TumbaLaCasaRemix,219392906,...,True,NaN,7858,public,"Alexio Ft. Daddy Yankee, Nicky Jam, Farruko y ...",single-track,154181977,2015-08-15T18:30:17Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW
9,https://i1.sndcdn.com/artworks-000017325329-xx...,NaN,https://soundcloud.com/erdemaktas/adele-someon...,2290,2012-01-25T19:04:45Z,NaN,100,284241,NaN,34517808,...,True,NaN,6257,public,Adele - Someone Like You,single-track,11182685,2012-01-25T19:04:45Z,eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJnZW8iO...,ALLOW


In [ ]:
browser.quit  #Tắt tab tạo ra